In [ ]:
# boxes 的形式为 （xmin, ymin, xmax, ymax）
class RandomResize(object):
    def __init__(self, sizes, max_size=None):
        assert isinstance(sizes, (list, tuple))
        self.sizes = sizes
        self.max_size = max_size

    def __call__(self, img, target=None):
        size = random.choice(self.sizes)
        return resize(img, target, size, self.max_size)


def resize(image, target, size, max_size=None):
    # 获取缩放后的图像尺寸
    size = get_size(image.size, size, max_size)
    # 对图像进行缩放
    rescaled_image = F.resize(image, size)

    if target is None:
        return rescaled_image, None

    # 计算 高的缩放比例 和 宽的缩放比例
    ratios = tuple(float(s) / float(s_orig) for s, s_orig in zip(rescaled_image.size, image.size))
    ratio_width, ratio_height = ratios

    target = target.copy()
    if "boxes" in target:
        boxes = target["boxes"]
        # 将 bbox 的高宽分别按照 高的缩放比例 和 宽的缩放比例 进行调整
        scaled_boxes = boxes * torch.as_tensor([ratio_width, ratio_height, ratio_width, ratio_height])
        target["boxes"] = scaled_boxes

    # 重新计算 bbox 的面积
    if "area" in target:
        area = target["area"]
        scaled_area = area * (ratio_width * ratio_height)
        target["area"] = scaled_area

    # 保存缩放后的高和宽
    h, w = size
    target["size"] = torch.tensor([h, w])

    if "masks" in target:
        target['masks'] = interpolate(
            target['masks'][:, None].float(), size, mode="nearest")[:, 0] > 0.5

    return rescaled_image, target


def get_size(image_size, size, max_size=None):
    if isinstance(size, (list, tuple)):
        return size[::-1]
    else:
        return get_size_with_aspect_ratio(image_size, size, max_size)


def get_size_with_aspect_ratio(image_size, size, max_size=None):
    w, h = image_size

    if max_size is not None:
        # 将size定为较短边长度，按照原始图像的长短比例（高宽比/宽高比），缩放图像较长边
        # 若缩放后图像的较长边大于最大尺寸 max_size， 则将较长边的长度定为 max_size，将
        # 较短边按照按照原始图像的比例进行调整
        min_original_size = float(min((w, h)))
        max_original_size = float(max((w, h)))
        if max_original_size / min_original_size * size > max_size:
            size = int(round(max_size * min_original_size / max_original_size))

    if (w <= h and w == size) or (h <= w and h == size):
        return (h, w)

    # 计算缩放后的图像高宽尺寸
    if w < h:
        ow = size
        oh = int(size * h / w)
    else:
        oh = size
        ow = int(size * w / h)

    return (oh, ow)


scales = [480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800]
img, target = RandomResize(scales, max_size=1333)(img, target)